In [ ]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, normalize

from utils.data_utils import mod_df, drop_features, inverse_mod_X, inverse_mod_y, fill_with_mean
# from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

In [ ]:
train_df = pd.read_csv(os.path.join('data', 'unionTrain.csv'))
test_df = pd.read_csv(os.path.join('data', 'unionTest.csv'))
print(f"Length Train: {len(train_df)}")
print(f"Length Test: {len(test_df)}")

In [ ]:
# print(len(train_df))
# train_df.head()

In [ ]:
# X_train_x = X_train.copy()
# X_train_x[X_train_x<=0] = pd.NA
# X_train_x.info()

In [ ]:
X_train, y_train = mod_df(train_df)
X_test, y_test = mod_df(test_df)

In [ ]:
def plot_nas(df: pd.DataFrame, ts=20):
    if df.isnull().sum().sum() != 0:
        na_df = (df.isnull().sum() / len(df)) * 100      
        na_df = na_df.drop(na_df[na_df == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio %' :na_df})
        # print(missing_data['Missing Ratio %'].to_numpy())
        print(f"Number of feature that lower than ts: {np.sum(missing_data['Missing Ratio %'].to_numpy()<=ts)}")
        missing_data.plot(kind = "barh")
        plt.axvline(ts, color="r")
        plt.figure(figsize=(8, 20))
        plt.show()
    else:
        print('No NAs found')

In [ ]:
def drop_id_timestep(df):
    result = df.copy()
    result = result.drop(['id', 'timestep'], axis=1)
    DEFAULT_FEATURE = result.columns.to_list()
    return result
X_train_1 = drop_id_timestep(X_train)
X_test_1 = drop_id_timestep(X_test)
y_train_1 = np.argmax(y_train.to_numpy(), axis=1)+1
y_test_1 = np.argmax(y_test.to_numpy(), axis=1)+1

In [ ]:
# plot ratio of missing value
X_train_x = X_train_1.copy()
X_train_x[X_train_x<=0] = pd.NA
plot_nas(X_train_x, ts=35)

In [ ]:
X_train_1['0X'].to_numpy().shape
DEFAULT_FEATURE = X_train_1.columns.to_list()
# y_train_1.shape

In [ ]:
def plot_feature_hist(pd_series, bins=100, feature_name=None):
    plt.hist(pd_series, bins=bins)
    plt.axvline(pd_series.mean(), color='red')
    plt.title(feature_name)
    plt.grid()
    plt.show()

In [ ]:
feature = '0Y'
X_train_1_feature = X_train_1[feature]
X_train_1_feature_mod = X_train_1_feature[X_train_1_feature>0]
print(np.sum(X_train_1[feature]<=0), np.sum(X_train_1[feature]>0), len(X_train_1))
print(np.sum(X_train_1[feature]<=0)/854, np.sum(X_train_1[feature]>0)/854, len(X_train_1)/854)
plot_feature_hist(X_train_1_feature_mod, bins=100, feature_name=feature)

In [ ]:
good_feature = []
p = 0.65
for feature in DEFAULT_FEATURE:
    # print(feature, np.sum(X_train_1[feature]<=0)>np.sum(X_train_1[feature]>0))
    if np.sum(X_train_1[feature]>0)/len(X_train_1[feature])>p:
        good_feature.append(feature)
    # print(np.sum(X_train_1[feature]<=0), np.sum(X_train_1[feature]>0), len(X_train_1))
print(len(good_feature), good_feature)

In [ ]:
def find_corr(ts=0.02):
    tmp = []
    for col in X_train_1.columns.to_list():
        corr = X_train_1[col].corr(pd.Series(y_train_1))
        # print(f"{col}\t{corr:f}\t{abs(corr):f}")
        if abs(corr) >= ts:
            tmp.append(col)
    # print(len(tmp), tmp)
    return tmp
# for i in np.arange(0.0, 0.22, 0.005):
#     print(f"ts:{i:.3f}\n{find_corr(i)}\nlength:{len(find_corr(i))}")
#     print("---------------------------------------------")

In [ ]:
tmp = []
ts = 0.001
for col in X_train_1.columns.to_list()[1::2]:
    tmp2 = []
    for col2 in X_train_1.columns.to_list()[1::2]:
        corr = X_train_1[col].corr( X_train_1[col2])
        tmp2.append(corr)
        # print(f"{col}\t{corr:f}\t{abs(corr):f}")
        # if corr >= ts:
            # tmp.append(col)
    tmp.append(tmp2)
print(len(tmp))
tmp = np.array(tmp)

In [ ]:
ts = 0.96
sns.heatmap(np.abs(tmp)>ts)
# print(list(tmp[0, :]))
for i in range(len(tmp)):
    print(i, list(np.where(tmp[i, :]>ts)))